In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DS&ML\\Wine-Quality-Prediction\\research'

In [3]:
os.chdir('../')
%pwd

'f:\\Files\\DS&ML\\Wine-Quality-Prediction'

In [4]:
from mlproject.constants import *

[2025-02-13 22:03:58,731 : INFO : __init__ : Logger has been set up successfully!]


In [5]:
from dataclasses import dataclass
from pathlib import Path
import os
import pandas as pd
import numpy as np
import joblib
import json
import mlflow
import dagshub
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import logging

In [6]:
from dataclasses import dataclass
import pandas as pd
import joblib
import json
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json
from mlproject import logger

In [7]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    test_data_path: Path
    all_params: dict
    metric_file_path: Path
    preprocessor_path: Path
    target_column: str

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config["artifacts_root"]])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.CatBoostClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_path=config.metric_file_path,
            preprocessor_path=config.preprocessor_path,
            target_column=schema.name,
        )
        return model_evaluation_config

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
        # Initialize MLflow tracking
        os.environ['MLFLOW_TRACKING_USERNAME'] = "JavithNaseem-J"
        os.environ['MLFLOW_TRACKING_PASSWORD'] = "fc8980ed3562ad9f63b1351811c99954c11dd06b"
        
        dagshub.init(repo_owner="JavithNaseem-J", repo_name="Wine-Quality-Prediction")
        mlflow.set_tracking_uri("https://dagshub.com/JavithNaseem-J/Wine-Quality-Prediction.mlflow")
        mlflow.set_experiment("wine_quality_prediction")

    def evaluate(self):
        try:

            with mlflow.start_run():
                # Set tags for the run
                mlflow.set_tag("model_type", "CatBoostClassifier")
                mlflow.set_tag("evaluation_stage", "testing")

                # Load preprocessor and model
                logger.info("Loading preprocessor and model...")
                preprocessor = joblib.load(self.config.preprocessor_path)
                model = joblib.load(self.config.model_path)

                # Log model parameters
                mlflow.log_params(self.config.all_params)

                # Load test data
                logger.info(f"Loading test data from {self.config.test_data_path}...")
                test_data = pd.read_csv(self.config.test_data_path)

                # Extract target column
                if self.config.target_column not in test_data.columns:
                    raise KeyError(f"Target column '{self.config.target_column}' not found in test data")

                test_y = test_data[self.config.target_column]
                test_x = test_data.drop(columns=[self.config.target_column])

                logger.info(f"Test data shape: X={test_x.shape}, y={test_y.shape}")

                # Preprocess test features
                logger.info("Preprocessing test features...")
                test_x_transformed = preprocessor.transform(test_x)

                # Make predictions
                logger.info("Making predictions on the test data...")
                predictions = model.predict(test_x_transformed)

                # Calculate and log metrics
                logger.info("Evaluating model performance...")
                metrics = {
                    "accuracy": accuracy_score(test_y, predictions),
                    "precision": precision_score(test_y, predictions, average="weighted"),
                    "recall": recall_score(test_y, predictions, average="weighted"),
                    "f1": f1_score(test_y, predictions, average="weighted")
                }

                # Log metrics to MLflow
                mlflow.log_metrics(metrics)

                # Log model with signature
                signature = mlflow.models.infer_signature(
                    test_x_transformed, predictions
                )
                mlflow.sklearn.log_model(
                    model,
                    "wine_quality_model",
                    signature=signature,
                    registered_model_name="WineQualityClassifier"
                )

                logger.info(f"Model Evaluation Metrics:\naccuracy: {metrics['accuracy']}\n"
                          f"precision: {metrics['precision']}\nrecall: {metrics['recall']}\n"
                          f"f1: {metrics['f1']}")

                # Save metrics locally
                metrics_path = os.path.join(self.config.root_dir, "metrics.json")
                with open(metrics_path, "w") as f:
                    json.dump(metrics, f, indent=4)
                logger.info(f"Evaluation metrics saved at {metrics_path}")

                return metrics

        except Exception as e:
            logger.error(f"Error in model evaluation: {str(e)}")
            raise e

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    metrics = model_evaluation.evaluate()

except FileNotFoundError as e:
    logger.error(f"File not found: {e}")
except KeyError as e:
    logger.error(f"Missing key in configuration or data: {e}")
except Exception as e:
    logger.error(f"Unexpected error: {e}")

[2025-02-13 22:04:03,520 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-02-13 22:04:03,532 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-02-13 22:04:03,535 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-02-13 22:04:03,535 : INFO : common : created directory at: artifacts]
[2025-02-13 22:04:03,535 : INFO : common : created directory at: artifacts/model_evaluation]


Accessing as JavithNaseem-J

Initialized MLflow to track repo "JavithNaseem-J/Wine-Quality-Prediction"

Repository JavithNaseem-J/Wine-Quality-Prediction initialized!

2025/02/13 22:04:05 INFO mlflow.tracking.fluent: Experiment with name 'wine_quality_prediction' does not exist. Creating a new experiment.


[2025-02-13 22:04:07,452 : INFO : 3856527910 : Loading preprocessor and model...]
[2025-02-13 22:04:08,599 : INFO : 3856527910 : Loading test data from artifacts/data_transformation/test.csv...]
[2025-02-13 22:04:08,637 : INFO : 3856527910 : Test data shape: X=(1022, 11), y=(1022,)]
[2025-02-13 22:04:08,637 : INFO : 3856527910 : Preprocessing test features...]
[2025-02-13 22:04:08,637 : INFO : 3856527910 : Making predictions on the test data...]
[2025-02-13 22:04:08,684 : INFO : 3856527910 : Evaluating model performance...]


Successfully registered model 'WineQualityClassifier'.
2025/02/13 22:04:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: WineQualityClassifier, version 1


[2025-02-13 22:04:28,270 : INFO : 3856527910 : Model Evaluation Metrics:
accuracy: 0.8336594911937377
precision: 0.826707337387785
recall: 0.8336594911937377
f1: 0.8250309018964447]
[2025-02-13 22:04:28,270 : INFO : 3856527910 : Evaluation metrics saved at artifacts/model_evaluation\metrics.json]


Created version '1' of model 'WineQualityClassifier'.


🏃 View run omniscient-bear-864 at: https://dagshub.com/JavithNaseem-J/Wine-Quality-Prediction.mlflow/#/experiments/1/runs/3bbe6aee33be4fd58e0a6f8529e07ab3
🧪 View experiment at: https://dagshub.com/JavithNaseem-J/Wine-Quality-Prediction.mlflow/#/experiments/1
